In [1]:
from riix.models.elo import Elo
from riix.utils import MatchupDataset, generate_matchup_data
from riix.metrics import binary_metrics_suite

In [2]:
df = generate_matchup_data() # replace with your pandas dataframe
dataset = MatchupDataset(
    df,
    competitor_cols=['competitor_1', 'competitor_2'],
    outcome_col='outcome',
    datetime_col='date',
    rating_period='1D',
)

loaded dataset with:
1000 matchups
100 unique competitors
10 rating periods of length 1D


In [3]:
model = Elo(num_competitors=dataset.num_competitors)
probs = model.rate_dataset(dataset)
metrics = binary_metrics_suite(probs=probs, outcomes=dataset.outcomes)


In [4]:
metrics

{'accuracy': 0.799,
 'log_loss': 0.5363807117304283,
 'brier_score': 0.1292005977180302}

In [5]:
model.print_top_k(k=5, competitor_names=dataset.competitors)

competitor   	rating
competitor_74	1730.661227
competitor_48	1727.693338
competitor_47	1725.183076
competitor_89	1719.935741
competitor_62	1712.183214
